In [1]:
! pip install faiss-gpu

In [9]:
import torch
import faiss
import numpy as np
from torchvision import datasets, transforms
from transformers import AutoImageProcessor, AutoModel
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from scipy.spatial.distance import chebyshev, minkowski
from tqdm import tqdm
from torch.utils.data import DataLoader
from collections import defaultdict
import pandas as pd
from torchvision import models as tv_models

models = {
    "ViT": "nateraw/vit-base-beans",
    "Swin": "microsoft/swin-base-patch4-window7-224",
    "ConvNeXT": "facebook/convnext-base-224",
    "RegNet": "facebook/regnet-y-040",
    "ResNet50": tv_models.resnet50,
    "ResNet101": tv_models.resnet101,
    "ResNet152": tv_models.resnet152,
    "DEiT": "facebook/deit-base-patch16-224",
    "BEiT": "microsoft/beit-base-patch16-224-pt22k-ft22k"
}

def load_model_and_processor(model_name):
    if model_name in ["ResNet50", "ResNet101", "ResNet152"]:
        model = models[model_name](pretrained=True)
        model = torch.nn.Sequential(*(list(model.children())[:-1])) 
        processor = None  
    else:
        processor = AutoImageProcessor.from_pretrained(models[model_name])
        model = AutoModel.from_pretrained(models[model_name])
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    return model, processor

def get_image_dataset(directory):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    dataset = datasets.ImageFolder(root=directory, transform=transform)
    return dataset

def get_embeddings(model, data_loader, device):
    embeddings = []
    labels = []
    for images, target in tqdm(data_loader):
        inputs = images.to(device)
        with torch.no_grad():
            outputs = model(inputs)
        if isinstance(outputs, torch.Tensor): 
            embedding = outputs.squeeze().cpu().numpy()
        else:
            embedding = outputs.pooler_output.cpu().numpy() if hasattr(outputs, "pooler_output") else outputs.last_hidden_state[:, 0].cpu().numpy()
        embeddings.append(embedding.reshape(len(images), -1))
        labels.extend(target.numpy())
    return np.vstack(embeddings), labels

def calculate_accuracy(distances, train_labels, test_labels, k=1):
    indices = np.argsort(distances, axis=1)[:, :k]
    correct = np.any([np.array([train_labels[idx] for idx in indices[:, i]]) == test_labels for i in range(k)], axis=0)
    return np.mean(correct) * 100

def calculate_person_wise_accuracy(distances, train_labels, test_labels, class_names, k=1):
    indices = np.argsort(distances, axis=1)[:, :k]
    correct = np.any([np.array([train_labels[idx] for idx in indices[:, i]]) == test_labels for i in range(k)], axis=0)
    person_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    
    for i, label in enumerate(test_labels):
        person_accuracy[class_names[label]]["total"] += 1
        if correct[i]:
            person_accuracy[class_names[label]]["correct"] += 1
    
    return person_accuracy

def format_accuracy_table(person_accuracy):
    data = {
        "Person": [],
        "Correct Predictions": [],
        "Total Predictions": [],
        "Accuracy (%)": []
    }
    
    for person, stats in person_accuracy.items():
        accuracy = (stats["correct"] / stats["total"]) * 100
        data["Person"].append(person)
        data["Correct Predictions"].append(stats["correct"])
        data["Total Predictions"].append(stats["total"])
        data["Accuracy (%)"].append(f"{accuracy:.2f}")
    
    df = pd.DataFrame(data)
    return df

def evaluate_distance_metrics(train_embeddings, test_embeddings, train_labels, test_labels, class_names):
    l2_distances = euclidean_distances(test_embeddings, train_embeddings)
    print(f"L2 Top-1 Accuracy: {calculate_accuracy(l2_distances, train_labels, test_labels, k=1):.2f}%")
    l2_person_accuracy = calculate_person_wise_accuracy(l2_distances, train_labels, test_labels, class_names, k=1)
    l2_table = format_accuracy_table(l2_person_accuracy)
    print("L2 Distance Person-wise Accuracy:")
    print(l2_table)

    l1_distances = manhattan_distances(test_embeddings, train_embeddings)
    print(f"L1 Top-1 Accuracy: {calculate_accuracy(l1_distances, train_labels, test_labels, k=1):.2f}%")
    l1_person_accuracy = calculate_person_wise_accuracy(l1_distances, train_labels, test_labels, class_names, k=1)
    l1_table = format_accuracy_table(l1_person_accuracy)
    print("L1 Distance Person-wise Accuracy:")
    print(l1_table)

    cosine_sim = cosine_similarity(test_embeddings, train_embeddings)
    print(f"Cosine Top-1 Accuracy: {calculate_accuracy(-cosine_sim, train_labels, test_labels, k=1):.2f}%")
    cosine_person_accuracy = calculate_person_wise_accuracy(-cosine_sim, train_labels, test_labels, class_names, k=1)
    cosine_table = format_accuracy_table(cosine_person_accuracy)
    print("Cosine Similarity Person-wise Accuracy:")
    print(cosine_table)

    cheb_distances = np.array([[chebyshev(test_emb, train_emb) for train_emb in train_embeddings] for test_emb in test_embeddings])
    print(f"Chebyshev Top-1 Accuracy: {calculate_accuracy(cheb_distances, train_labels, test_labels, k=1):.2f}%")
    cheb_person_accuracy = calculate_person_wise_accuracy(cheb_distances, train_labels, test_labels, class_names, k=1)
    cheb_table = format_accuracy_table(cheb_person_accuracy)
    print("Chebyshev Distance Person-wise Accuracy:")
    print(cheb_table)

    mink_distances = np.array([[minkowski(test_emb, train_emb, p=3) for train_emb in train_embeddings] for test_emb in test_embeddings])
    print(f"Minkowski Top-1 Accuracy: {calculate_accuracy(mink_distances, train_labels, test_labels, k=1):.2f}%")
    mink_person_accuracy = calculate_person_wise_accuracy(mink_distances, train_labels, test_labels, class_names, k=1)
    mink_table = format_accuracy_table(mink_person_accuracy)
    print("Minkowski Distance Person-wise Accuracy:")
    print(mink_table)

    faiss_index = faiss.IndexFlatL2(train_embeddings.shape[1])
    faiss_index.add(train_embeddings.astype('float32'))
    _, faiss_indices = faiss_index.search(test_embeddings.astype('float32'), k=3)
    
    faiss_acc = np.mean([train_labels[i] == test_labels[j] for j, i in enumerate(faiss_indices[:, 0])]) * 100
    print(f"FAISS (L2) Top-1 Accuracy: {faiss_acc:.2f}%")
        
    faiss_person_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    for i, test_label in enumerate(test_labels):
        faiss_person_accuracy[class_names[test_label]]["total"] += 1
        if train_labels[faiss_indices[i, 0]] == test_label:
            faiss_person_accuracy[class_names[test_label]]["correct"] += 1

    faiss_table = format_accuracy_table(faiss_person_accuracy)
    print("FAISS (L2) Person-wise Accuracy:")
    print(faiss_table)

def main(model_name):
    model, processor = load_model_and_processor(model_name)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    train_dataset = get_image_dataset("DATASET-COLLECTED/train")
    test_dataset = get_image_dataset("DATASET-COLLECTED/test")

    class_names = train_dataset.classes

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    train_embeddings, train_labels = get_embeddings(model, train_loader, device)
    test_embeddings, test_labels = get_embeddings(model, test_loader, device)

    evaluate_distance_metrics(train_embeddings, test_embeddings, train_labels, test_labels, class_names)

In [ ]:
if __name__ == "__main__":
    model_name = "ViT"
    main(model_name)


In [3]:
if __name__ == "__main__":
    model_name = "ConvNeXT"
    main(model_name)

100%|██████████| 127/127 [00:09<00:00, 13.04it/s]


L2 Top-1 Accuracy: 29.92%
Person person-10: 20.00% (2/10)
Person person-11: 50.00% (5/10)
Person person-12: 50.00% (4/8)
Person person-13: 0.00% (0/9)
Person person-14: 0.00% (0/3)
Person person-15: 100.00% (8/8)
Person person-2: 0.00% (0/10)
Person person-3: 20.00% (2/10)
Person person-4: 55.56% (5/9)
Person person-5: 30.00% (3/10)
Person person-6: 9.09% (1/11)
Person person-7: 36.36% (4/11)
Person person-8: 10.00% (1/10)
Person person-9: 37.50% (3/8)
L1 Top-1 Accuracy: 31.50%
Person person-10: 30.00% (3/10)
Person person-11: 50.00% (5/10)
Person person-12: 50.00% (4/8)
Person person-13: 0.00% (0/9)
Person person-14: 0.00% (0/3)
Person person-15: 100.00% (8/8)
Person person-2: 0.00% (0/10)
Person person-3: 20.00% (2/10)
Person person-4: 55.56% (5/9)
Person person-5: 30.00% (3/10)
Person person-6: 9.09% (1/11)
Person person-7: 45.45% (5/11)
Person person-8: 10.00% (1/10)
Person person-9: 37.50% (3/8)
Cosine Top-1 Accuracy: 29.92%
Person person-10: 20.00% (2/10)
Person person-11: 50.00%

In [4]:
if __name__ == "__main__":
    model_name = "Swin"
    main(model_name)

100%|██████████| 127/127 [00:14<00:00,  8.51it/s]


L2 Top-1 Accuracy: 27.56%
Person person-10: 20.00% (2/10)
Person person-11: 30.00% (3/10)
Person person-12: 25.00% (2/8)
Person person-13: 11.11% (1/9)
Person person-14: 0.00% (0/3)
Person person-15: 62.50% (5/8)
Person person-2: 20.00% (2/10)
Person person-3: 0.00% (0/10)
Person person-4: 66.67% (6/9)
Person person-5: 50.00% (5/10)
Person person-6: 0.00% (0/11)
Person person-7: 36.36% (4/11)
Person person-8: 30.00% (3/10)
Person person-9: 25.00% (2/8)
L1 Top-1 Accuracy: 29.13%
Person person-10: 20.00% (2/10)
Person person-11: 30.00% (3/10)
Person person-12: 25.00% (2/8)
Person person-13: 11.11% (1/9)
Person person-14: 0.00% (0/3)
Person person-15: 62.50% (5/8)
Person person-2: 30.00% (3/10)
Person person-3: 0.00% (0/10)
Person person-4: 66.67% (6/9)
Person person-5: 50.00% (5/10)
Person person-6: 0.00% (0/11)
Person person-7: 36.36% (4/11)
Person person-8: 40.00% (4/10)
Person person-9: 25.00% (2/8)
Cosine Top-1 Accuracy: 32.28%
Person person-10: 20.00% (2/10)
Person person-11: 30.00%

In [5]:
if __name__ == "__main__":
    model_name = "RegNet"
    main(model_name)

100%|██████████| 127/127 [00:11<00:00, 11.52it/s]


L2 Top-1 Accuracy: 22.05%
Person person-10: 10.00% (1/10)
Person person-11: 10.00% (1/10)
Person person-12: 0.00% (0/8)
Person person-13: 11.11% (1/9)
Person person-14: 0.00% (0/3)
Person person-15: 100.00% (8/8)
Person person-2: 20.00% (2/10)
Person person-3: 0.00% (0/10)
Person person-4: 11.11% (1/9)
Person person-5: 50.00% (5/10)
Person person-6: 18.18% (2/11)
Person person-7: 36.36% (4/11)
Person person-8: 10.00% (1/10)
Person person-9: 25.00% (2/8)
L1 Top-1 Accuracy: 22.05%
Person person-10: 10.00% (1/10)
Person person-11: 10.00% (1/10)
Person person-12: 0.00% (0/8)
Person person-13: 0.00% (0/9)
Person person-14: 0.00% (0/3)
Person person-15: 100.00% (8/8)
Person person-2: 30.00% (3/10)
Person person-3: 0.00% (0/10)
Person person-4: 11.11% (1/9)
Person person-5: 50.00% (5/10)
Person person-6: 0.00% (0/11)
Person person-7: 54.55% (6/11)
Person person-8: 10.00% (1/10)
Person person-9: 25.00% (2/8)
Cosine Top-1 Accuracy: 25.20%
Person person-10: 20.00% (2/10)
Person person-11: 10.00%

In [1]:
import torch
import faiss
import numpy as np
from torchvision import datasets, transforms
from transformers import AutoImageProcessor, AutoModel
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from scipy.spatial.distance import chebyshev, minkowski
from tqdm import tqdm
from torch.utils.data import DataLoader
from collections import defaultdict
import pandas as pd
from torchvision import models as tv_models

models = {
    "ViT": "nateraw/vit-base-beans",
    "Swin": "microsoft/swin-base-patch4-window7-224",
    "ConvNeXT": "facebook/convnext-base-224",
    "RegNet": "facebook/regnet-y-040",
    "ResNet50": tv_models.resnet50,
    "ResNet101": tv_models.resnet101,
    "ResNet152": tv_models.resnet152,
    "DEiT": "facebook/deit-base-patch16-224",
}

def load_model_and_processor(model_name):
    if model_name in ["ResNet50", "ResNet101", "ResNet152"]:
        model = models[model_name](pretrained=True)
        model = torch.nn.Sequential(*(list(model.children())[:-1]))
        processor = None
    else:
        processor = AutoImageProcessor.from_pretrained(models[model_name])
        model = AutoModel.from_pretrained(models[model_name])
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    return model, processor

def get_image_dataset(directory):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    dataset = datasets.ImageFolder(root=directory, transform=transform)
    return dataset

def get_embeddings(model, data_loader, device):
    embeddings = []
    labels = []
    for images, target in tqdm(data_loader):
        inputs = images.to(device)
        with torch.no_grad():
            outputs = model(inputs)
        if isinstance(outputs, torch.Tensor):
            embedding = outputs.squeeze().cpu().numpy()
        else:
            embedding = outputs.pooler_output.cpu().numpy() if hasattr(outputs, "pooler_output") else outputs.last_hidden_state[:, 0].cpu().numpy()
        embeddings.append(embedding.reshape(len(images), -1))
        labels.extend(target.numpy())
    return np.vstack(embeddings), labels

def calculate_accuracy(distances, train_labels, test_labels, k=1):
    indices = np.argsort(distances, axis=1)[:, :k]
    correct = np.any([np.array([train_labels[idx] for idx in indices[:, i]]) == test_labels for i in range(k)], axis=0)
    return np.mean(correct) * 100

def calculate_top_k_accuracy(distances, train_labels, test_labels, k):
    indices = np.argsort(distances, axis=1)[:, :k]
    correct_top_k = np.any([np.array([train_labels[idx] for idx in indices[:, i]]) == test_labels for i in range(k)], axis=0)
    return np.mean(correct_top_k) * 100

def calculate_person_wise_accuracy(distances, train_labels, test_labels, class_names, k=1):
    indices = np.argsort(distances, axis=1)[:, :k]
    correct = np.any([np.array([train_labels[idx] for idx in indices[:, i]]) == test_labels for i in range(k)], axis=0)
    person_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    
    for i, label in enumerate(test_labels):
        person_accuracy[class_names[label]]["total"] += 1
        if correct[i]:
            person_accuracy[class_names[label]]["correct"] += 1
    
    return person_accuracy

def format_accuracy_table(person_accuracy):
    data = {
        "Person": [],
        "Correct Predictions": [],
        "Total Predictions": [],
        "Accuracy (%)": []
    }
    
    for person, stats in person_accuracy.items():
        accuracy = (stats["correct"] / stats["total"]) * 100
        data["Person"].append(person)
        data["Correct Predictions"].append(stats["correct"])
        data["Total Predictions"].append(stats["total"])
        data["Accuracy (%)"].append(f"{accuracy:.2f}")
    
    df = pd.DataFrame(data)
    return df

def evaluate_distance_metrics(model_name, train_embeddings, test_embeddings, train_labels, test_labels, class_names, results):
    l2_distances = euclidean_distances(test_embeddings, train_embeddings)
    top_1_acc = calculate_accuracy(l2_distances, train_labels, test_labels, k=1)
    top_3_acc = calculate_accuracy(l2_distances, train_labels, test_labels, k=3)
    top_k_acc = calculate_top_k_accuracy(l2_distances, train_labels, test_labels, k=5)

    results.append({
        "Model": model_name,
        "Top-1 Accuracy (%)": top_1_acc,
        "Top-3 Accuracy (%)": top_3_acc,
        "Top-5 Accuracy (%)": top_k_acc
    })

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_dataset = get_image_dataset("DATASET-COLLECTED/train")
    test_dataset = get_image_dataset("DATASET-COLLECTED/test")
    class_names = train_dataset.classes

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    results = []

    for model_name in models.keys():
        print(f"Evaluating model: {model_name}")
        model, processor = load_model_and_processor(model_name)
        
        train_embeddings, train_labels = get_embeddings(model, train_loader, device)
        test_embeddings, test_labels = get_embeddings(model, test_loader, device)
        
        evaluate_distance_metrics(model_name, train_embeddings, test_embeddings, train_labels, test_labels, class_names, results)

    results_df = pd.DataFrame(results)
    print("All Models Evaluation Results:")
    print(results_df)

if __name__ == "__main__":
    main()


Evaluating model: ViT


Some weights of ViTModel were not initialized from the model checkpoint at nateraw/vit-base-beans and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 127/127 [00:06<00:00, 18.42it/s]


Evaluating model: Swin


100%|██████████| 127/127 [00:12<00:00,  9.90it/s]


Evaluating model: ConvNeXT


...

100%|██████████| 127/127 [00:09<00:00, 12.98it/s]
/home/utkarsh/miniconda3/envs/pytorch_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/utkarsh/miniconda3/envs/pytorch_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Evaluating model: ResNet101


100%|██████████| 127/127 [00:09<00:00, 12.94it/s]
/home/utkarsh/miniconda3/envs/pytorch_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/utkarsh/miniconda3/envs/pytorch_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Evaluating model: ResNet152


 98%|█████████▊| 124/127 [00:10<00:00, 11.28it/s]